# Check loss function

Compare to official implementation as in `deps/loss.py`

In [1]:
%load_ext autoreload
%autoreload 2

import sys; sys.path.append("..")

# Load

In [2]:
import mlx.core as mx
import tensorflow as tf
import numpy as np

In [3]:
from model import load_model_tokenizer
from qa import build_parser, load_processed_datasets, test, loss_fn, compute_loss
from deps.loss import compute_official_bert_squad_loss, mx_to_tf

/Users/paul/Envs/PyTorTenFloHuffinFace/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
parser = build_parser()
args = parser.parse_args("")
args.weights_finetuned = '../weights/final_fine_tuned_full_data_1000_iter.npz'
args.dataset_size = 100

In [5]:
model, tokenizer = load_model_tokenizer(
    hf_model=args.model_str, weights_finetuned_path=args.weights_finetuned)

/Users/paul/Envs/PyTorTenFloHuffinFace/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
train_ds, valid_ds, test_ds = load_processed_datasets(tokenizer, args.dataset_size)

Map: 100%|███████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 1717.29 examples/s]


In [7]:
test(model, valid_ds, 10)

Test loss 0.054, Test ppl 1.056, Test eval took 0.409s


In [8]:
dataset = valid_ds
batch_size = 1

for s in range(0, len(dataset), batch_size):
    batch = dataset[s: s + batch_size]
    input_ids, token_type_ids, attention_mask, start_positions, end_positions = map(
        mx.array,
        (batch['input_ids'], batch['token_type_ids'], batch['attention_mask'],
         batch['start_positions'], batch['end_positions'])
    )
    losses = loss_fn(model, input_ids, token_type_ids, attention_mask,
                     start_positions, end_positions, reduction="none")

    print(losses)
    # official implementation
    start_logits, end_logits = model(input_ids=input_ids, token_type_ids=token_type_ids,
                                     attention_mask=attention_mask)
    loss = compute_loss(start_logits, end_logits, start_positions, end_positions, reduction="none")
    print(loss)
    start_logits, end_logits, start_positions, end_positions = map(
        mx_to_tf,
        (start_logits, end_logits, start_positions, end_positions)
    )
    seq_length = input_ids.shape[1]
    losses = compute_official_bert_squad_loss(start_logits, end_logits, start_positions, end_positions,
                                              seq_length)
    print(losses)

array(0.59311, dtype=float32)
array([0.59311], dtype=float32)
tf.Tensor(0.5931101, shape=(), dtype=float32)
array(0.318884, dtype=float32)
array([0.318884], dtype=float32)
tf.Tensor(0.3188846, shape=(), dtype=float32)
array(0.0625553, dtype=float32)
array([0.0625553], dtype=float32)
tf.Tensor(0.062555194, shape=(), dtype=float32)
array(1.63804, dtype=float32)
array([1.63804], dtype=float32)
tf.Tensor(1.638041, shape=(), dtype=float32)
array(0.406376, dtype=float32)
array([0.406376], dtype=float32)
tf.Tensor(0.4063757, shape=(), dtype=float32)
array(1.97606, dtype=float32)
array([1.97606], dtype=float32)
tf.Tensor(1.9760616, shape=(), dtype=float32)
array(0.0223765, dtype=float32)
array([0.0223765], dtype=float32)
tf.Tensor(0.022376684, shape=(), dtype=float32)
array(0.116567, dtype=float32)
array([0.116567], dtype=float32)
tf.Tensor(0.11656732, shape=(), dtype=float32)
array(0.0808032, dtype=float32)
array([0.0808032], dtype=float32)
tf.Tensor(0.08080314, shape=(), dtype=float32)
array